In [79]:
pip install -U langchain-community pinecone-client openai tiktoken ipywidgets -U langchain-openai astor


Note: you may need to restart the kernel to use updated packages.


In [161]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
import ast
import astor
import openai


In [151]:
# Load the environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

EMBEDDINGS_SIZE_LARGE = 3072
EMBEDDINGS_SIZE_SMALL = 1536

In [173]:
def get_file_content(file_path):
    with open(file_path, 'r') as file:
        print(file.name.split('/')[-1])
        return file.read()

In [164]:
client = openai.Client(api_key=OPENAI_API_KEY)

def summarize_file(file_path):
    data = get_file_content(file_path)
    completion = client.chat.completions.create(
      model="gpt-4-turbo",
      messages=[
          {"role": "system", "content": "You are an assistant trained to summarize code files succinctly. When given a file, provide a brief summary without any introductions or additional text."},
          {"role": "user", "content": "Please summarize the following file:\n\n" + data}
      ]
    )
    
    return completion.choices[0].message.content

In [174]:
file = get_file_content("./Smarderobe/server/app/routers/user_router.py")

user_router.py


In [166]:
file

'from app.database.schemas import UserSchema, UserOutSchema, ResetPasswordSchema, ResetPasswordRequestScheam\nfrom sqlalchemy import select, insert, delete, update, exc\nfrom sqlalchemy.orm import Session, selectinload\nfrom app.db_setup import get_db\nfrom fastapi import Request, Depends, APIRouter, HTTPException, status\nfrom app.database.models import User, Profile\nfrom app.auth import get_password_hash, get_user_id, create_access_token,  SECRET_KEY, ALGORITHM, oauth2_scheme, credentials_exception, ACCESS_TOKEN_EXPIRE_MINUTES\nfrom app.logging.logger import logger\nfrom typing import Annotated\nfrom app.send_email import  send_verification_email\nfrom app.config import backend_base_url, frontend_base_url\nfrom datetime import timedelta\nfrom app.send_email import env\nfrom fastapi.templating import Jinja2Templates\nfrom fastapi.responses import HTMLResponse\nfrom jose import JWTError, jwt\n\nrouter = APIRouter()\n\ntemplates = Jinja2Templates(directory="./app/templates")\n\n@router

In [169]:
summary = summarize_file("./Smarderobe/server/app/routers/user_router.py")

In [170]:
summary

'The file defines API endpoints for a user management system using FastAPI, handling user creation, listing, validation, password reset requests, refreshing tokens, and deletion through various routes. It interacts with a database using SQLAlchemy to perform operations like adding a new user, getting user details, and verifying user emails. The endpoints use schemas for input validation, send emails for verification and password resets, and handle exceptions appropriately, providing responses based on the outcome of the operations. Additionally, token-based verification is employed for email verification and password reset functionalities. The system logs errors and uses templates for email responses.'

In [138]:
import ast
import astor
import os

class CodeVisitor(ast.NodeVisitor):
    def __init__(self):
        self.code_elements = []

    
    # def visit_Import(self, node):
    #     self.elements.append(node)  # Store the AST node directly

    # def visit_ImportFrom(self, node):
    #     self.elements.append(node)  # Store the AST node directly

    # def visit_ClassDef(self, node):
    #     self.elements.append(node)  # Store the AST node directly

    # # If you want to process body items as well
    # def visit_Assign(self, node):
    #     self.elements.append(node)  # Store assignment nodes

    # def visit_Expr(self, node):
    #     self.elements.append(node)  # Store expression nodes

    # def visit_Return(self, node):
    #     self.elements.append(node)  # Store return nodes

    # def visit_FunctionDef(self, node):
    #     self.elements.append(node)  # Store the AST node directly

    def visit_FunctionDef(self, node):
        # Store function source code and metadata
        self.code_elements.append({
            "page_content": astor.to_source(node),
            "metadata": {
                "type": "function",
                "name": node.name
            }
        })

    def visit_ClassDef(self, node):
        # Store the class itself
        self.code_elements.append({
            "page_content": astor.to_source(node),
            "metadata": {
                "type": "class",
                "name": node.name
            }
        })
        # Also visit methods within the class
        method_visitor = CodeVisitor()
        for n in node.body:
            if isinstance(n, ast.FunctionDef):
                method_visitor.visit(n)
        self.code_elements.extend(method_visitor.code_elements)

def parse_code_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        source_code = file.read()
    tree = ast.parse(source_code)
    visitor = CodeVisitor()
    visitor.visit(tree)
    return [
        {**element, "metadata": {**element["metadata"], "source": os.path.normpath(file_path)}}
        for element in visitor.code_elements
    ]

def traverse_directory(directory):
    all_code_elements = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                normalized_path = os.path.normpath(file_path)
                # get summaries about the file content from chatGPT
                file_summary =         {
                    "page_content": summarize_file(file_path),
                    "metadata": {
                        "source": normalized_path,
                        "type": "summary",
                        "name":  normalized_path.split('\\')[-1]
                    }
                }
                all_code_elements.append(file_summary)
                code_elements = parse_code_from_file(file_path)
                all_code_elements.extend(code_elements)
    return all_code_elements

# Example usage
project_directory = "./Smarderobe/server"
all_code_details = traverse_directory(project_directory)
    

In [179]:
def traverse_directory(directory):
    for root, dirs, files in os.walk(directory):
        print(root, dirs, files)
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                # get summaries about the file content from chatGPT
                file_summary =         {
                    # "page_content": summarize_file(file_path),
                    "metadata": {
                        "source": os.path.normpath(file_path),
                        "type": "summary",
                        "name":  os.path.normpath(file_path).split('\\')[-1]
                    }
                }
                print(f"file summary: {file_summary}")
                code_elements = parse_code_from_file(file_path)


# Example usage
traverse_directory("./Smarderobe/server")


./Smarderobe/server ['alembic', 'app', 'static'] ['.gitignore', 'alembic.ini', 'main.py', 'requirements.txt', 'run.py']
file summary: {'metadata': {'source': 'Smarderobe\\server\\main.py', 'type': 'summary', 'name': 'main.py'}}
file summary: {'metadata': {'source': 'Smarderobe\\server\\run.py', 'type': 'summary', 'name': 'run.py'}}
./Smarderobe/server\alembic ['versions'] ['env.py', 'README', 'script.py.mako']
file summary: {'metadata': {'source': 'Smarderobe\\server\\alembic\\env.py', 'type': 'summary', 'name': 'env.py'}}
./Smarderobe/server\alembic\versions [] ['31ed8a88e96c_update_countries_clothing_items_tables.py', '3eab37b818e9_check.py', '5f630b6a9222_initial_migration.py', '8cd4ecbd3e80_add_color_bridge.py']
file summary: {'metadata': {'source': 'Smarderobe\\server\\alembic\\versions\\31ed8a88e96c_update_countries_clothing_items_tables.py', 'type': 'summary', 'name': '31ed8a88e96c_update_countries_clothing_items_tables.py'}}
file summary: {'metadata': {'source': 'Smarderobe\\se

In [ ]:
def get_gpt_files_descriptions():
    pass

In [ ]:
all_files_descriptions = get_gpt_files_descriptions()

In [129]:
embeddings = OpenAIEmbeddings( model ="text-embedding-3-large", openai_api_key=OPENAI_API_KEY)


In [130]:
# Function to initialize a FAISS vector store from documents
def create_faiss_library(documents):
    texts = [d['page_content'] for d in documents]
    metadatas = [d['metadata'] for d in documents]
    # Adjust the following line if the FAISS library API differs
    return FAISS.from_texts(texts,  metadatas=metadatas, embedding=embeddings)

In [139]:
library = create_faiss_library(all_code_details)


In [147]:
query = "user class"

In [148]:
query_answer = library.similarity_search(query, 5)

In [142]:
print(query_answer[0])

page_content="@router.get('/verification/{token}', response_class=HTMLResponse)\ndef validate_user(token: str, request: Request, db: Session=Depends(get_db)):\n    try:\n        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])\n        user_id: int = payload.get('id')\n        query = select(User).where(User.id == user_id)\n        db_user = db.scalars(query).first()\n        if not db_user:\n            raise HTTPException(status_code=404, detail='User not found')\n        db_user.email_verified = True\n        db.commit()\n        return templates.TemplateResponse(request=request, name=\n            'account_verified.html', context={'message':\n            'Your account has been verified.'})\n    except JWTError:\n        return templates.TemplateResponse(request=request, name=\n            'account_verified.html', context={'message': 'Token is not valid.'}\n            )\n    except Exception:\n        return templates.TemplateResponse(request=request, name=\n        

In [149]:
print(query_answer[1].page_content)

class UserOutSchema(BaseModel):
    first_name: str = Field(..., min_length=1, max_length=100)
    last_name: str = Field(..., min_length=1, max_length=100)
    email: EmailStr = Field(..., min_length=1, max_length=50, description=
        'Email has to be Unique and is required.')
    email_verified: bool
    is_active: bool
    id: int
    profile: Optional[ProfileSchema]



In [150]:
for answer in query_answer:
    print(answer.metadata)
    print(answer.metadata['source'])
    print(answer.metadata['name'])
    print("\n")

{'type': 'class', 'name': 'User', 'source': 'Smarderobe\\server\\app\\database\\models\\user.py'}
Smarderobe\server\app\database\models\user.py
User


{'type': 'class', 'name': 'UserOutSchema', 'source': 'Smarderobe\\server\\app\\database\\schemas\\user_schema.py'}
Smarderobe\server\app\database\schemas\user_schema.py
UserOutSchema


{'type': 'class', 'name': 'UserLoginSchema', 'source': 'Smarderobe\\server\\app\\database\\schemas\\user_schema.py'}
Smarderobe\server\app\database\schemas\user_schema.py
UserLoginSchema


{'type': 'function', 'name': 'validate_user', 'source': 'Smarderobe\\server\\app\\routers\\user_router.py'}
Smarderobe\server\app\routers\user_router.py
validate_user


{'type': 'class', 'name': 'UserSchema', 'source': 'Smarderobe\\server\\app\\database\\schemas\\user_schema.py'}
Smarderobe\server\app\database\schemas\user_schema.py
UserSchema




In [152]:
query2 = "upload image"

In [153]:
query2_answer = library.similarity_search(query2, 2)

In [154]:
print(query2_answer[0].metadata['source'])
print(query2_answer[0].page_content)

Smarderobe\server\app\routers\clothing_item_router.py
def upload_image(file: UploadFile, IMAGEDIR: str) ->str:
    filename = f"{uuid4()}.{file.filename.split('.')[-1]}"
    file_path = os.path.join(IMAGEDIR, filename)
    with open(file_path, 'wb') as f:
        for chunk in file.file:
            f.write(chunk)
    return filename



In [155]:
for answer in query2_answer:
    print(answer.metadata["source"])
    print(answer.page_content)
    print("\n")

Smarderobe\server\app\routers\clothing_item_router.py
def upload_image(file: UploadFile, IMAGEDIR: str) ->str:
    filename = f"{uuid4()}.{file.filename.split('.')[-1]}"
    file_path = os.path.join(IMAGEDIR, filename)
    with open(file_path, 'wb') as f:
        for chunk in file.file:
            f.write(chunk)
    return filename



Smarderobe\server\app\routers\clothing_item_router.py
@router.post('/', status_code=201)
def add_clothing_item(user_id: Annotated[int, Depends(get_user_id)], name:
    str=Form(...), description: str=Form(None), colours: str=Form(...),
    size: str=Form(...), type: str=Form(None), brand_id: int=Form(...),
    file: UploadFile=File(...), db: Session=Depends(get_db)):
    if file.filename.split('.')[-1].lower() not in ['jpg', 'jpeg', 'bmp',
        'webp', 'png']:
        raise HTTPException(status_code=400, detail=
            'Image extension is not supported')
    type_id = db.scalars(Select(Type.id).where(Type.name == type)).first()
    clothing_item_

In [66]:
print(query2_answer[1].metadata['source'])
print(query2_answer[1].page_content)

Smarderobe\server\app\routers\clothing_item_router.py
def list_clothing_items(db): Expression: Constant(value='\n    Get all clothing items\n    ') Assign: Name(id='result', ctx=Store()) = Call(func=Attribute(value=Call(func=Attribute(value=Name(id='db', ctx=Load()), attr='scalars', ctx=Load()), args=[Call(func=Attribute(value=Call(func=Name(id='Select', ctx=Load()), args=[Name(id='ClothingItem', ctx=Load())], keywords=[]), attr='options', ctx=Load()), args=[Call(func=Name(id='selectinload', ctx=Load()), args=[Attribute(value=Name(id='ClothingItem', ctx=Load()), attr='item_images', ctx=Load())], keywords=[])], keywords=[])], keywords=[]), attr='all', ctx=Load()), args=[], keywords=[]) If Return: Name(id='result', ctx=Load()) def list_clothing_items(clothing_id, db): Expression: Constant(value='\n    Get one clothing item based on its id\n    ') Assign: Name(id='result', ctx=Store()) = Call(func=Attribute(value=Call(func=Attribute(value=Name(id='db', ctx=Load()), attr='scalars', ctx=Loa

In [156]:
query3 = "get single user details"

In [157]:
query3_answer = library.similarity_search(query3, 2)

In [158]:
print(query3_answer[0].metadata['source'])
print(query3_answer[0].page_content)

Smarderobe\server\app\routers\user_router.py
@router.get('/{user_id}', status_code=200)
def list_users(user_id: int, db: Session=Depends(get_db)):
    """
    Fetches one user based on user_id
    """
    result = db.scalars(select(User).where(User.id == user_id).options(
        selectinload(User.profile))).first()
    if not result:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=
            'No users login credentials found')
    return result



In [159]:
for answer in query3_answer:
    print(answer.metadata["source"])
    print(answer.page_content)
    print("\n")

Smarderobe\server\app\routers\user_router.py
@router.get('/{user_id}', status_code=200)
def list_users(user_id: int, db: Session=Depends(get_db)):
    """
    Fetches one user based on user_id
    """
    result = db.scalars(select(User).where(User.id == user_id).options(
        selectinload(User.profile))).first()
    if not result:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=
            'No users login credentials found')
    return result



Smarderobe\server\app\routers\user_router.py
@router.get('/', status_code=200)
def list_users(db: Session=Depends(get_db)):
    """
    Fetches all users
    """
    result = db.scalars(select(User).options(selectinload(User.profile))).all()
    if not result:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=
            'No users login credentials found')
    return result



